In [8]:
import pandas as pd
from rapidfuzz import process, fuzz

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import logging as transformers_logging
transformers_logging.set_verbosity_error()

class NERParser:
    def __init__(self, model_name: str = "dslim/bert-base-NER", lowercase: bool = False):
        """
        Initialize the NER parser with a model and optionally configure the lowercase preprocessing.
        """
        self.model_name = model_name
        self.lowercase = lowercase
        self.device = self.get_device()
        
        # Load the tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, do_lower_case=self.lowercase)
        self.model = AutoModelForTokenClassification.from_pretrained(self.model_name)
        
        # Set up the NER pipeline
        self.nlp_pipeline = pipeline("ner", 
                                     model=self.model, 
                                     tokenizer=self.tokenizer, 
                                     device=self.device, 
                                     aggregation_strategy="simple")

    def get_device(self):
        """
        Determines whether to use MPS, CUDA, or CPU depending on the available hardware.
        """
        if torch.backends.mps.is_available():
            print("MPS device found, using MPS backend.\n")
            return torch.device("mps")
        elif torch.cuda.is_available():
            print(f"CUDA device found, using CUDA backend. Device: {torch.cuda.get_device_name(0)}\n")
            return torch.device("cuda")
        else:
            print("Neither MPS nor CUDA found, using CPU.\n")
            return torch.device("cpu")

    
    def parse_ner_results(self, ner_results: list):
        """
        Parse the NER results and extract entities related to 'PER' (persons) and 'MISC' (potential movie titles).
        """
        per_entities, misc_entities = [], []
        
        for entity in ner_results:
            # Extraction of all Persons
            if entity['entity_group'] == 'PER':
                per_entities.append(entity['word'])
            # Extraction of all Misc that could indicate movies
            elif entity['entity_group'] == 'MISC':
                misc_entities.append(entity['word'])
        
        return per_entities, misc_entities

    
    def process_query(self, query: str):
        """
        Processes a text query, runs NER, and returns the extracted actors and movie names.
        """
        # Optionally lowercase the input if configured
        if self.lowercase:
            query = query.lower()
        
        # Run the NER pipeline
        ner_results = self.nlp_pipeline(query)

        # Parse the results to extract actors and movies
        per_entities, misc_entities = self.parse_ner_results(ner_results)
        
        return per_entities, misc_entities

ner_parser = NERParser(lowercase=False)

Neither MPS nor CUDA found, using CPU.



In [10]:
db = pd.read_pickle('exports/graph.pkl')

In [11]:
query = "When was Angelina Jolie born and where? Did she play in Titanic?"
person, movies = ner_parser.process_query(query)

In [12]:
context_person = []

for p in person:
    # Exact Lookup
    relevant = db[db.index == p].dropna(axis=1)
    
    if not relevant.empty:
        context_person.append(relevant.iloc[0])
    
    else:
        # Levenshtein matching for closest match
        matches = process.extractOne(p, db.index, scorer=fuzz.ratio)
        
        if matches and matches[1] > 80:
            closest_match = matches[0]
            relevant = db[db.index == closest_match].dropna(axis=1)
            context_person.append(relevant.iloc[0])
   
context_df_person = pd.DataFrame(context_person)

context_movie = [] 
   
for m in movies:
    # Exact Lookup
    relevant = db[db.index == m].dropna(axis=1)
    
    if not relevant.empty:
        context_movie.append(relevant.iloc[0])
    
    else:
        # Levenshtein matching for closest match
        matches = process.extractOne(m, db.index, scorer=fuzz.ratio)
        
        if matches and matches[1] > 80:
            closest_match = matches[0]
            relevant = db[db.index == closest_match].dropna(axis=1)
            context_movie.append(relevant.iloc[0])   
        
context_df_movie = pd.DataFrame(context_movie)

In [13]:
context_df_movie

Predicate,Australian Classification,CNC film rating (France),FSK film rating,Filmiroda rating,Hong Kong film rating,ICAA rating,IMDb ID,JMK film rating,MTRCB rating,Medierådet rating,...,nominated for,original broadcaster,original language of film or TV show,performer,production company,production designer,publication date,"references work, tradition or theory",screenwriter,set in period
Titanic,M,"no age restriction, no age restriction","FSK 12, FSK 12, FSK 6",Category II,Category IIA,not recommended for children under 12,"tt0036443, tt0115392, tt0120338, tt0046435",free from 10 years,PG,For ages 11 and up,...,"Academy Award for Best Film Editing, Academy A...",CBS,"English, German, English, English",James Horner,"Universum Film AG, 20th Century Studios, 20th ...","Maurice Ransford, Peter Lamont","1953-01-01, 1997-11-01, 1943-01-01, 1996-01-01",Come Josephine in My Flying Machine,"Herbert Selpin, James Cameron, Walter Reisch, ...","1990s, 1912"


In [14]:
context_df_person.columns

Index(['IMDb ID', 'ancestral home', 'award received', 'country of citizenship',
       'described by source', 'father', 'image', 'instance of',
       'languages spoken, written or signed', 'mother', 'native language',
       'node description', 'node label', 'nominated for', 'notable work',
       'occupation', 'place of birth', 'relative', 'religion', 'residence',
       'sibling', 'significant event', 'spouse', 'unmarried partner'],
      dtype='object', name='Predicate')

In [19]:
import requests

url = 'http://nlp.stanford.edu/data/glove.6B.zip'
response = requests.get(url)

with open('glove.6B.zip', 'wb') as file:
    file.write(response.content)

print('Download complete!')


Download complete!


In [16]:
!unzip glove.6B.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
import numpy as np

def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Load the GloVe embeddings (300-dimensional)
glove_embeddings = load_glove_embeddings("glove.6B.300d.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.300d.txt'

In [43]:
def embed_phrase(phrase, embeddings):
    words = phrase.split()
    word_vectors = [embeddings[word.lower()] for word in words if word.lower() in embeddings]
    
    if len(word_vectors) == 0:
        return np.zeros(300)  # Return zero vector if no words have embeddings
    
    return np.mean(word_vectors, axis=0)

# Embed column names of knowledge graph db
column_embeddings = {col: embed_phrase(col, glove_embeddings) for col in context_df_person.columns}

In [44]:
column_embeddings

{'IMDb ID': array([ 0.24651998,  0.07036   , -0.14642501,  0.116624  ,  0.056493  ,
        -0.22114   , -0.09043   , -0.075065  ,  0.00623   ,  0.04271001,
         0.137815  ,  0.33861   ,  0.440625  ,  0.16979034, -0.35968   ,
         0.061608  , -0.174365  , -0.03683   , -0.22728   , -0.42122   ,
         0.03673501,  0.04362   , -0.528735  , -0.296985  ,  0.07916079,
         0.24066399,  0.120575  ,  0.462405  ,  0.18459   , -0.377335  ,
        -0.31038502, -0.0232805 , -0.04938   ,  0.31717497,  0.005455  ,
        -0.235765  , -0.265915  , -0.0691155 , -0.132442  , -0.470625  ,
         0.28242052, -0.412005  ,  0.32108   ,  0.62975   , -0.30144352,
        -0.18836328,  0.036367  , -0.484735  , -0.39131   , -0.63302004,
         0.0996885 ,  0.139675  ,  0.49345   , -0.405625  ,  0.026585  ,
        -0.12878   ,  0.1299315 , -0.229645  , -0.50788   , -0.2958    ,
        -0.173519  ,  0.2378495 , -0.11698   ,  0.134761  , -0.13264701,
        -0.10877001, -0.159163  ,  0.019

In [45]:
# Embed the query keywords
query = "When was Angelina Jolie born and where? Did she play in Titanic?"
query_embeddings = [embed_phrase(word, glove_embeddings) for word in query]


In [46]:
query_embeddings

[array([-0.58186  ,  0.24353  , -0.53713  ,  0.018981 , -0.81685  ,
         0.020117 , -0.341    ,  0.84737  ,  0.14889  , -0.51869  ,
         0.19258  ,  0.14905  , -0.51267  ,  0.48473  , -0.31117  ,
         0.044387 , -0.45924  , -0.1054   ,  0.60693  , -0.12483  ,
        -0.53615  , -0.47126  , -0.082498 ,  0.13124  ,  0.21124  ,
         0.34524  , -0.54644  ,  0.9673   ,  0.18186  ,  0.048264 ,
         0.30786  ,  0.40401  , -0.19185  ,  0.29347  , -0.65749  ,
        -0.18127  , -0.063808 , -0.51852  , -0.059752 ,  0.19683  ,
        -0.42032  ,  0.25235  , -0.83283  ,  0.80818  , -0.40206  ,
        -0.42321  ,  0.45571  , -0.12721  , -0.064699 ,  0.081907 ,
         0.49701  ,  0.078343 ,  0.34832  , -0.25249  , -0.21429  ,
         0.14305  ,  0.47827  , -0.19347  ,  0.022003 , -0.35109  ,
         0.81247  ,  0.98627  , -0.33525  ,  0.21812  , -0.0027489,
        -0.090565 , -0.036789 , -0.063974 ,  0.52971  , -0.68823  ,
        -0.10498  ,  0.85387  ,  0.21031  ,  0.2

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_sim(vec1, vec2):
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0 
    
    return np.dot(vec1, vec2) / (norm_vec1 * norm_vec2)

def find_closest_columns(query_embeddings, column_embeddings, top_n=5):
    column_similarities = {}

    for col, col_vec in column_embeddings.items():
        similarities = [cosine_sim(col_vec, q_vec) for q_vec in query_embeddings if np.linalg.norm(q_vec) > 0]
        column_similarities[col] = np.mean(similarities) if similarities else -1

    sorted_columns = sorted(column_similarities.items(), key=lambda item: item[1], reverse=True)
    
    return [col for col, sim in sorted_columns[:top_n]]

top_columns = find_closest_columns(query_embeddings, column_embeddings, top_n=5)

print("Top columns:", top_columns)

Top columns: ['instance of', 'languages spoken, written or signed', 'described by source', 'place of birth', 'node label']


In [58]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [49]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.38s/it]


In [55]:
query = "When was Angelina Jolie born?"
columns = [c.replace(" ", "") for c in top_columns]
prompt = f"Query: {query}\n\nColumns:\n{', '.join(columns)}\n\nWhich columns are relevant to answer the query?"

inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=30)

relevant_columns = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Relevant Columns:", relevant_columns)

Relevant Columns: Query: When was Angelina Jolie born?

Columns:
instanceof, languagesspoken,writtenorsigned, describedbysource, placeofbirth, nodelabel

Which columns are relevant to answer the query?

Response:The relevant columns to answer the query about Angelina Jolie's birth date are "placeofbirth" and "
